In [191]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import scale, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score
from scipy import stats
import copy 

## Задание 1. Линейная регрессия 

Прочитаем данные из файла, приведем к нужному формату, избавимся от лишних столбцов и некорректных значений. 

In [192]:

data = pd.read_csv("men_shoes.csv")
# drop text columns
data = data.drop(["Brand_Name", "Product_details"], axis=1)
# drop rows with missing values
data = data.dropna()
# prepare data 
data["How_Many_Sold"] = data["How_Many_Sold"].str.replace(",", "").astype(int)
data["Current_Price"] = data["Current_Price"].str.replace(",", "").str.replace("₹", "").astype(float)

data.head()

,How_Many_Sold,Current_Price,RATING
0,2242,1098.0,3.8
1,240,674.0,4.0
2,16662,588.0,3.8
3,135,599.0,4.0
4,240,982.0,4.0


Построим корреляционную матрицу, чтобы убедиться в том, что независимые переменные не коррелируют между собой.  

In [193]:
correlation_matrix = data.corr()
print(correlation_matrix)

               How_Many_Sold  Current_Price    RATING
How_Many_Sold       1.000000      -0.155577  0.144131
Current_Price      -0.155577       1.000000  0.469463
RATING              0.144131       0.469463  1.000000


In [194]:
X, y = data[["How_Many_Sold", "Current_Price"]], data["RATING"]
X = scale(X)

### Модель линейной регрессии 

$$ 
f(X) = X \cdot \omega + \varepsilon = \omega_0 x_0 + \omega_1 x_1 + ... + \omega_n x_n + \varepsilon
$$
где $X$ - матрица признаков, $\omega$ - вектор весов, $\varepsilon$ - случайная ошибка, $f(x)$ - вектор прогнозов. 

При этом, так как ошибки распределены нормально: 
$$ 
E(\varepsilon) = 0, \quad D(\varepsilon) = \sigma^2 I 
$$
И отсутствует мультиколлинеарность признаков, то оценка весов $\omega$ будет оптимальной (в смысле минимизации среднеквадратичной ошибки). 

Выбор весов $\omega$ заключается в минимизации функции потерь, например, среднеквадратичной ошибки -- $J(\omega, x, y)$ 

$$
J(\omega, x, y) = \frac{1}{2n} \sum_{i=1}^{n} (f(x_i) - y_i)^2 =  \frac{1}{2n} \sum_{i=1}^{n}(\omega^T x_i - y_i)^2 = \frac{1}{2n} (\omega^T X^T X \omega - 2 \omega^T X^T y + y^T y)
$$

$$ 
\frac{\partial J}{\partial \omega} = 0 \Rightarrow \omega = (X^T X)^{-1} X^T y
$$

При этом предсказания модели будут равны:
$$
\hat{y} = X \cdot \omega
$$

Таким образом, веса $\omega$ можно найти вершив матричное уравнение. Напишем класс для работы с линейной регрессией. 


In [195]:
class LinearRegression(object):
    def fit(self, X, y):
        X = np.insert(X, 0, 1, axis=1)  # add ones vector (free coefficients, aka bias)
        temp = np.linalg.inv(X.T @ X)  
        weights = np.linalg.multi_dot([temp, X.T, y])  
        self.bias, self.weights = weights[0], weights[1:]

    def get_weights(self):
        return self.weights
    
    def zero_weights(self):
        self.weights = np.zeros(self.weights.shape)

    def get_bias(self):
        return self.bias
    
    def predict(self, X):
        return X @ self.weights + self.bias

Найдем веса $\omega$ на основе обучающей выборки (с помощью метода fit) 

In [196]:
linear_regression = LinearRegression()
linear_regression.fit(X, y)

print("Weights:", linear_regression.get_weights())
print("Bias:", linear_regression.get_bias())

Weights: [0.09004836 0.20395935]
Bias: 3.8206896551724143


Найдем оценку остаточной дисперсии $\overline{\sigma^2}$ по формуле:
$$
\overline{\sigma^2} = \frac{RSS}{n - m} = \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{n - m}
$$
где $n$ - количество наблюдений, $m$ - количество признаков (включая свободный член), $y_i$ - истинное значение, $\hat{y}_i$ - предсказанное значение.

Для этого найдем предсказания модели на тестовой выборке (с помощью метода predict) и посчитаем остаточную дисперсию. 

In [197]:
def residual_variance(n, m, y_true, y_hat):
    numerator = np.sum((y_true - y_hat) ** 2)
    return numerator / (n - m) 

In [198]:
y_hat = linear_regression.predict(X) 

r_var = residual_variance(len(y), len(linear_regression.get_weights()), y, y_hat)
print("Residual variance:", r_var)

Residual variance: 0.11972680388060147


Вычислим коэффициент детерминации $R^2$ по формуле: 
$$
R^2 = 1 - \frac{RSS}{TSS} = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \overline{y})^2}
$$
Значение этого коэффициента показывает какую долю дисперсии зависимой переменной объясняют независимые переменные.

In [199]:
def R2_score(y_true, y_hat):
    numerator = np.sum((y_true - y_hat) ** 2)
    denominator = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - numerator / denominator

In [200]:
r2_score = R2_score(y, y_hat)
print("R2 score:", r2_score)

R2 score: 0.26872791897563864


### Поиск доверительных интервалов 

Так как случайные ошибки распределены нормально, то предсказания модели также будут распределены нормально.

$$ 
\hat{y} \sim N(X \cdot \omega, \sigma^2 I)
$$
На основании этого можно сделать вывод, что веса $\omega$ также распределены нормально. 
$$ 
\hat{\omega} \sim N(\omega, \sigma^2 (X^T X)^{-1})
$$

$$ 
\frac{\hat{\omega} - \omega}{\sigma(\hat{\omega})} = \frac{\hat{\omega} - \omega}{\sqrt{\sigma^2 (X^tX)^{-1}}} \sim t(n - m)
$$

Можно построить доверительные интервалы для весов $\omega$ 
$$ 
-t_{1 - \alpha/2} \leq \frac{\hat{\omega} - \omega}{\sqrt{\sigma^2 (X^tX)^{-1}}} \leq t_{1 - \alpha/2}
$$
$$ 
\hat{\omega} - t_{1 - \alpha/2} \sqrt{\sigma^2 (X^tX)^{-1}} \leq \omega \leq \hat{\omega} + t_{1 - \alpha/2} \sqrt{\sigma^2 (X^tX)^{-1}}
$$
где $t_{1 - \alpha/2}$ - квантиль распределения Стьюдента уровня $1 - \alpha/2$ с $n - m$ степенями свободы.

In [201]:
alpha = 0.025

n = len(y)
m = len(linear_regression.get_weights()) # number of features
t = stats.t.ppf(1 - alpha / 2, df=n - m)

left_omega = linear_regression.get_weights() - t * np.sqrt(r_var * np.diag(np.linalg.inv(X.T @ X)))
right_omega = linear_regression.get_weights() + t * np.sqrt(r_var * np.diag(np.linalg.inv(X.T @ X)))

left_bias = linear_regression.get_bias() - t * np.sqrt(r_var * np.linalg.inv(X.T @ X)[0, 0])
right_bias = linear_regression.get_bias() + t * np.sqrt(r_var * np.linalg.inv(X.T @ X)[0, 0])

print(f"Confidence interval for weights:")
for i, (l, r) in enumerate(zip(left_omega, right_omega)):
    print(f"Weight {i}: [{l}, {r}], hat value = {linear_regression.get_weights()[i]}")
    
print(f"Confidence interval for bias: [{left_bias}, {right_bias}], hat value = {linear_regression.get_bias()}")

    

Confidence interval for weights:
Weight 0: [0.08488700163345049, 0.09520971620831252], hat value = 0.0900483589208815
Weight 1: [0.19879798911582108, 0.20912070369068309], hat value = 0.20395934640325208
Confidence interval for bias: [3.8155282978849834, 3.8258510124598453], hat value = 3.8206896551724143


## Оценка гипотез 

### Чем больше продажи, тем больше рейтинг 

Сформулируем гипотезу:
$$
\begin{array}{ll}
H_0: \omega_0 = 0 &\text{продажи не влияют на рейтинг}\\
H_1: \omega_0 \neq 0& \text{продажи влияют на рейтинг}
\end{array}
$$

Статистика $t$ для этой гипотезы будет равна: 
$$
t = \frac{\hat{\omega}_0}{\sqrt{\sigma^2 (X^tX)^{-1}_{00}}}
$$

In [202]:
t_stat = (linear_regression.get_weights()[0]) / np.sqrt(r_var * np.linalg.inv(X.T @ X)[0, 0])
print("t-statistic:", t_stat)
critical_value = stats.t.ppf(1 - alpha / 2, df=n - m)
print("Critical value:", critical_value)

if t_stat > critical_value:
    print("H0 is rejected")
else:
    print("H0 is not rejected")

t-statistic: 39.107498756530546
Critical value: 2.241548609204152
H0 is rejected


Так как значение статистики $t$ оказалось больше критического значения, то гипотеза $H_0$ отвергается. Таким образом, продажи влияют на рейтинг. 

### Рейтинг зависит от цены 
Сформулируем гипотезу:
$$
\begin{array}{ll}
H_0: \omega_1 = 0 &\text{цена не влияет на рейтинг}\\
H_1: \omega_1 \neq 0& \text{цена влияет на рейтинг}
\end{array}
$$

In [203]:
t_stat = (linear_regression.get_weights()[1]) / np.sqrt(r_var * np.linalg.inv(X.T @ X)[1, 1])
print("t-statistic:", t_stat)
critical_value = stats.t.ppf(1 - alpha / 2, df=n - m)
print("Critical value:", critical_value)

if t_stat > critical_value:
    print("H0 is rejected")
else:
    print("H0 is not rejected")

t-statistic: 88.57840366481479
Critical value: 2.241548609204152
H0 is rejected


Цена влияет на рейтинг, так как значение статистики $t$ оказалось больше критического значения.

### Коэффициенты при цене и продажах одновременно равны нулю 
Сформулируем гипотезу:
$$
\begin{array}{ll}
H_0: \omega_0 = \omega_1 = 0 \\ 
H_1: \overline{H_0}& 
\end{array}
$$
Будем рассматривать модель с нулевыми коэффициентами при цене и продажах и полную модель. 

Будем использовать F-тест для проверки этой гипотезы. 
$$
F = \frac{(RSS_0 - RSS)/(m - k)}{RSS/(n - m)} \sim F_{m - k, n - m}
$$

Для случая, когда мы проверяем гипотезу при всех признаках равных нулю формула статистики выглядит следующим образом:

$$F =\frac{R_2^2}{1-R_2^2}\cdot\frac{n-m}{m} \sim F_{n, n-m}$$

In [204]:
f_stat = r2_score ** 2 / (1 - r2_score ** 2) * (n - m) / (m)
print("F-statistic:", f_stat)
critical_value = stats.f.ppf(1 - alpha, m, n - m)
print("Critical value:", critical_value)

if f_stat > critical_value:
    print("H0 is rejected")
else:
    print("H0 is not rejected")

F-statistic: 900.5574992684265
Critical value: 3.689467581943152
H0 is rejected


Нулевая гипотеза отвергается, так как значение статистики $F$ оказалось больше критического значения. 

## Задание 2 


Проведем однофакторный дисперсионный анализ для проверки гипотезы о зависимости индекса массы тела от наличия привычки к курению. 
Суть дисперсионного анализа заключается в том, что мы разбиваем выборку на несколько групп и сравниваем дисперсии двух видов -- межгрупповую и внутригрупповую. 

Выдвинем гипотезу: 
$$
\begin{array}{ll}
H_0: \text{индекс массы тела не зависит от привычки к курению}\\
H_1: \text{индекс массы тела зависит от привычки к курению}
\end{array}
$$

Ее статистика $F$ будет равна:
$$
F = \frac{S_{model}/\text{df}_{model}}{S_{error}/\text{df}_{error}} \sim F_{\text{df}_{model}, \text{df}_{error}}
$$

$$
S_{\text{total}} = S_{\text{model}} + S_{\text{error}} + S_{\text{const}} 
$$ 
$S_{\text{total}}$ - общая сумма квадратов, $S_{\text{model}}$ - сумма квадратов модели, $S_{\text{error}}$ - остаточная сумма квадратов, $S_{\text{const}}$ - сумма квадратов константы.

In [210]:
data = pd.read_csv("sex_bmi_smokers.csv")
data.drop("sex", axis=1, inplace=True)
data.head()

,bmi,smoker
0,27.900,yes
1,33.770,no
2,33.000,no
3,22.705,no
4,28.880,no


In [226]:
smokers = data[data["smoker"] == "yes"]
non_smokers = data[data["smoker"] == "no"]

total_mean = data["bmi"].mean() 
smokers_diff = smokers["bmi"] - total_mean
non_smokers_diff = non_smokers["bmi"] - total_mean

smokers_diff_mean = smokers_diff.mean()
non_smokers_diff_mean = non_smokers_diff.mean()

smokers_error = (smokers_diff - smokers_diff_mean)
non_smokers_error =(non_smokers_diff - non_smokers_diff_mean)



49719.501037119255
